NOTE: If running for the first time uncomment and run the last line of this notebook first.

In [3]:
import pandas as pd

In [4]:
pickle_base = '../../data/pickle/'
tissue      = 'LUSC'
directory   = tissue + '-PAIRED/'

# Paired experiments
TCGA   = {"LUSC": ['43-6771', '18-5595', '22-5471', '39-5031', '33-4589', '33-4586'],
          "LUAD": ['50-5931', '15-5420', '38-4631', '49-4488', '50-5932', '50-5935'],
          "COAD": ['AA-3697', 'AA-3713', 'AA-3506', 'AZ-6599', 'A6-2686', 'A6-2682'],
          "BRCA": ['BH-A0BZ', 'BH-A1EN', 'BH-A1F5', 'E2-A1IO'],          
          "GBM":  ['74-6573', '06-0152']}
    
minor       =  TCGA['LUSC'][0][:2]
major       =  TCGA['LUSC'][0][3:]

anno_str       = 'Illumina-450k-Anno.{rev}.{ext}'.format(rev='hg19',ext='pkl')
file_str_picl  = 'TCGA-{minor}-{major}-{tissue}.{ext}'.format(minor=minor,
                                                                major=major,
                                                                tissue=tissue,
                                                                ext='pkl')

filepath = pickle_base + file_str_picl
filepath

'../../data/pickle/TCGA-43-6771-LUSC.pkl'

In [5]:
def slice_df_by_chrom(df, fields):
    '''
    Slice a dataframe by chromosome.

    :param probe: A dataframe object loaded from l1base_hsfli_8438.xlsx file.
    :return: A dictionary of L1 locations keyed by chromosome. 
    '''
    chr_positions = {}
    for chr in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,20,22,'X','Y']: # chr number
        chr_positions[chr] = df.loc[df['Chr']==chr][fields] # slice df
    return chr_positions

In [6]:
anno = Annotate_Methyl_450k(filepath)

In [7]:
l1base_df = pd.read_excel('../../data/l1base/l1base_hsfli_8438.xlsx')

l1base_list = []
for index, row in l1base_df.iterrows():
    l1base_list.append([row[0], row[1], row[2], row[5], row[6], row[7], row[8], row[9]])

headers = ['Chr','Start','End','Strand','ORF1 Start','ORF1 End','ORF2 Start','ORF2 End']
l1base_df = pd.DataFrame(l1base_list, columns=headers)

In [8]:
# include the following headings in the sliced object
headings = ['Start', 'End', 'ORF1 Start', 'ORF1 End', 'ORF2 Start', 'ORF2 End']

chr_dict = slice_df_by_chrom(l1base_df, headings)

ch='X' # ----- Note: this sets 'ch' for the entire noteboook. 

print ('Active L1s in chromosome {n}: '.format(n=ch))
L1chr = chr_dict[ch]
L1chr

Active L1s in chromosome X: 


,Start,End,ORF1 Start,ORF1 End,ORF2 Start,ORF2 End
132,11706247,11714295,1908,2921,2988,6812
133,23237515,23245560,1907,2920,2987,6811
134,142476843,142484883,1909,2919,2986,6810
135,155523050,155515003,1909,2922,2989,6813
136,141428248,141420202,1908,2921,2988,6812
137,96064845,96056798,1909,2922,2989,6813
138,83066639,83058591,1909,2922,2989,6813
139,64020288,64012236,1909,2922,2989,6813
140,54125747,54117700,1909,2922,2989,6813
141,47790701,47782657,1909,2922,2989,6810


In [9]:
gene_probes = anno.get_probe_refs_by_gene('NFE2L2')
for probe in gene_probes:
    print (anno.get_probe(probe).hg38_cord, 
           anno.get_probe(probe).GDC_CGI_Coordinate,
           round(anno.get_probe(probe).Beta_Normal, 2),
           round(anno.get_probe(probe).Beta_Tumor, 2),
          ) 

177264045 chr2:177263545-177265119 0.01 0.01
177265557 chr2:177263545-177265119 0.02 0.03
177265176 chr2:177263545-177265119 0.08 0.08
177265102 chr2:177263545-177265119 0.02 0.03
177264271 chr2:177263545-177265119 0.01 0.02
177264953 chr2:177263545-177265119 0.01 0.03
177265172 chr2:177263545-177265119 0.06 0.07
177230898 chr2:177212457-177213487 0.94 0.95
177264039 chr2:177263545-177265119 0.01 0.01
177265185 chr2:177263545-177265119 0.03 0.04
177265388 chr2:177263545-177265119 0.04 0.05
177264609 chr2:177263545-177265119 0.07 0.1
177243172 chr2:177263545-177265119 0.92 0.81


In [155]:
l1base_df = pd.read_excel('../../data/l1base/l1base_hsfli_8438.xlsx')

l1base_list = []
for index, row in l1base_df.iterrows():
    l1base_list.append([row[0], row[1], row[2], row[5], row[6], row[7], row[8], row[9]])

headers = ['Chr','Start','End','Strand','ORF1 Start','ORF1 End','ORF2 Start','ORF2 End']
l1base_df = pd.DataFrame(l1base_list, columns=headers)

In [2]:
# %load '../methylator/annotation/annotate_methyl_450k.py'
import os
import _pickle as pickle

class Probe:
    """
    Holds Illumina 450k probe data with GDC data for a single probe loci.
    """
    def __init__(self):
        self.id = None          # REF_ID (Illumina)
        self.name = None        # Name (Illumina)
        self.chr = None         # Chromosome (Illumina)
        self.gene = None        # UCSC_RefGene_Name (Illumina)
        self.refseq = None      # UCSC_RefGene_Accession (Illumina)
        self.strand = None      # Strand orientation (-/+) (Illumina)
        self.seq = None         # Sequence prior to bi-sulfite conv. (Illumina)
        self.feature = None             # Defines a UCSC_RefGene_Group feature. (Illumina)
        self.tour = None                # Location of CpG relative to CpG island (UCSC_CpG_Island).
        self.hg19_cord = None           # Coordinate of probe in hg19. (Ilumina)
        self.hg38_cord = None           # Coordinate of probe in hg38. (GDC)
        self.GDC_CGI_Coordinate = None  # 
        self.GDC_gene = None            # Gene name. (GDC)
        self.GDC_gene_type = None       # Gene function i.e. protein-coding, etc.
        self.Beta_Normal =  None        # Methylation beta-value for normal sample.
        self.Beta_Tumor = None         # Methylation beta-value for tumor sample. 

class Interval:
    """
    Define a genomic interval by chromsome and orientation.
    """
    def __init__(self, chromosome, start, end, strand):
        self.chr = chromosome
        self.start = start
        self.end = end
        self.strand = strand

class UCSC_RefGene_Group:
    """
    Defines a UCSC_RefGene_Group feature.
        • TSS200 = 0–200 bases upstream of TSS.
        • TSS1500 = 200–1500 bases upstream of TSS.
        • 5'UTR = Between TSS and the ATG start site, in UTR.
        • Body = Between ATG and stop codon.
        • 3'UTR = Between the stop codon and poly A signal.
    """
    BODY    = "Body"
    TSS200  = "TSS200"
    TSS1500 = "TSS1500"
    UTR5   = "5'UTR"
    UTR3   = "3'UTR"
    EXON   = "Exon"

class UCSC_CpG_Island:
    """
    Defines relation to UCSC CpG island.
        • Shore = 0–2 kb from island.
        • Shelf = 2–4 kb from island.
        • N = upstream (5’) of CpG island.
        • S = downstream (3’) of CpG island.
    """
    ISLAND = "Island"
    NSHORE = "N_Shore"
    SSHORE = "S_Shore"
    NSHELF = "N_Shelf"
    SSHELF = "S_Shelf"
    
class Annotate_Methyl_450k(object):
    """
    Load Illumina probe information along with a single paired GDC Experiment.
    """

    def __init__(self, filepath):
        try:
            with open(filepath, 'rb') as f:
                self.probe = pickle.load(f)
        except (FileNotFoundError, IOError):
            pass

    def get_number(self):
        """
        Return total number of probes.
        """
        number = 0
        for probe_id in self.probe.keys():
            number += 1

        return number

    def get_probe(self, ref_id):
        """
        Return probe associated with a ref_id.
        """
        try:
            probe = self.probe[ref_id]
        except Exception as ex:
            probe = None
            print("WARNING: No probe with ref_id of %s found." % ref_id)
        return probe
    
    def get_all_probes(self):
        """
        Return list of all probes.
        """
        probe_list = []
        for probe in self.probe.keys():
            probe_list.append(self.get_probe(probe))
        return probe_list
    
    def get_probes_by_list(self, list_of_ids):
        """
        Return a list of probes from a list of references.
        """
        out_list = []
        for probe_id in list_of_ids:
            out_list.append(self.get_probe(probe_id))

        return out_list
    
    def get_probe_refs_by_gene(self, gene_name):
        """
        Get all probes associated with a gene.
        """
        probes = {k: self.probe[k] for k in self.probe 
                  if gene_name in self.probe[k].gene}
        return self.get_keys(probes.keys())

    def get_keys(self, dict_keys):
        """
        Get Probe ref_id keys.
        """
        l = []
        for i in dict_keys:
            l.append(i)
        return l
    
    def get_probe_refs_by_location(self, probe_loc):
        """
        Get all probe references associated with a genomic location.
        """
        probes = {k: self.probe[k] for k in self.probe if probe_loc in self.probe[k].loc}
        return self.get_keys(probes.keys())

    def get_probes_by_cpg(self, cpg_loc):
        """
        Get a list probes within a cpg location.
        """
        pass

    def get_probes_by_chr(self, chr_loc):
        """
        Get a list of probes within a certain genomic region
        """
        pass

    def get_probes_by_chr_and_loc(self, chr_loc):
        """
        Get a list of probes within a certain genomic region
        """
        pass

    def get_refid_by_chr_and_loc(self, chr_loc):
        """
        Get a list of probe reference *keys* within a genomic region
        """
        pass

    def get_hg18_coord(self, probe):
        """
        Get genomic coordinate of a single probe.
        """
        return probe.hg18_cord

    def get_hg38_coord(self, probe):
        """
        Get genomic coordinate of a single probe.
        """
        return probe.hg38_cord
    
    def get_sorted_probes_by_refid(self):
        """
        Sort probes according to probe id.
        """
        sorted_keys = sorted(list(self.probe.keys()))
        return sorted_keys
    
    def get_sorted_probes_by_chr(self):
        """
        Sort probes according to probe id.
        """
        return sorted(self.get_all_probes(), key=lambda x: x.chr)

In [ ]:
# This is not the last line------------------------------------------

l = ['cg00000029','cg00000165','cg00000289','cg00000363','cg00000658'] #list of probe refs

print( 'Number of probes reported by \'get_number\': {a}'.format(
    a=anno.get_number()) )
print( 'Probe object returned from \'get_probe\': {a}'.format(
    a=anno.get_probe('cg00000029')))
print( 'Length of \'get_all_probes\' call: {a}'.format(
    a=len(anno.get_all_probes())) )
print( 'Number of probes returned by \'get_probes_by_list\': {a}'.format(
    a=len(anno.get_probes_by_list(l))) )
print( 'Get probes associated with a gene (Illumina): {a}'.format(
    a=anno.get_probe_refs_by_gene('TP53')) ) 
print( 'CGI coordinate (GDC): {a}'.format(
    a=anno.probe.get('cg00000029').GDC_CGI_Coordinate) )
print( 'Gene type (GDC): {a}'.format(
    a=anno.probe.get('cg00000029').GDC_gene_type) )
print( 'Hg19 coordinate (GDC): {a}'.format(
    a=anno.probe.get('cg00000029').hg19_cord) )
print( 'Hg38 coordinate (GDC): {a}'.format(
    a=anno.probe.get('cg00000029').hg38_cord) )